# 🧫 🧪 Эксперименты с классическими *unsupervised* методами обнаружения аномалий

### 🌐 Установка [pyod](https://github.com/yzhao062/pyod)

In [ ]:
# !pip3 install pyod

## 💅 Предобработка данных

In [ ]:
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from gensim.utils import tokenize
import nltk; nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


def strip_html_tags(text):
    """Удаление html tags из текста."""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def preprocess_text(text):
    text = strip_html_tags(text)  # удаление html tags
    text = strip_non_alphanum(text) # заменили все небуквенные символы на пробел
    text = strip_numeric(text) # удалили все цифры
    text = remove_stopwords(text) # удалили все стоп-слова
    # text = strip_short(text, minsize=2) # удалили короткие слова
    word_list = list(tokenize(text, deacc=True, to_lower=True)) # токенизация, deacc - избавляет от ударений
    word_list = [WordNetLemmatizer().lemmatize(word) for word in word_list] # лемматизация
    return ' '.join(word for word in word_list)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pavelmamaev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 🧬 Экперименты

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

import pyod
from pyod.models import iforest
from pyod.models import lof
from pyod.models import knn
from pyod.models import copod

c = 0.1  # отношение количества аномальных экземпляров к нормальным

categories = ["comp.graphics",
              'talk.politics.mideast',
              "rec.sport.hockey",
              "sci.med",
              "sci.space",
              'misc.forsale',
              'soc.religion.christian',
              'talk.politics.misc']

experimant_cnt = 0
all_experiments = len(categories) * (len(categories) -  1)
auc_list_iforest = []
auc_list_lof = []
auc_list_knn = []
auc_list_copod = []

# Формирование словаря с категориями
dataset = {}
for cat in categories:
    # Загрузка
    dataset[cat] = fetch_20newsgroups(subset='all', categories=[cat],
                            shuffle=True, random_state=123,
                            remove=('headers', 'footers'), return_X_y=True)[0]
    # Предобработка
    dataset[cat] = [preprocess_text(text) for text in dataset[cat]]

# Перебираем пары категорий
for i in range(len(categories)):
    for j in range(len(categories)):
        if i == j:
            continue

        c1 = categories[i]
        c2 = categories[j]

        experimant_cnt += 1

        # Формирование нормальной и аномальной выборок
        normal_data = dataset[c1]
        anomal_data = dataset[c2][:min(int(c * len(normal_data)) + 1, len(dataset[c2]))]
        all_data = normal_data + anomal_data

        # TF-IDF векторизация
        vectorizer = TfidfVectorizer()
        all_data_tf = vectorizer.fit_transform(all_data).toarray()

        # Формирование выборок
        x = all_data_tf
        y = np.array([False] * len(normal_data) + [True] * len(anomal_data))
        all_data, x, y = shuffle(all_data, x, y, random_state=123)

        # Задаем модели

        # Isolation Forest
        iforest_clf = iforest.IForest(
            contamination=0.1,
            n_estimators=3000,
            max_samples=1.0,
            random_state=123,
            n_jobs=-1,
        )

        # LOF
        lof_clf = lof.LOF(
            contamination=0.1,
            n_neighbors=2,
            metric='canberra',
            n_jobs=-1,
        )

        #kNN
        knn_clf = knn.KNN(
            contamination=0.1,
            n_neighbors=2,
            method='mean',
            metric='l1',
            n_jobs=-1,
        )

        # COPOD
        copod_clf = copod.COPOD(contamination=0.1)

        # Считаем метрику ROC AUC
        auc_iforest = iforest_clf.fit_predict_score(x, y)
        auc_lof = lof_clf.fit_predict_score(x, y)
        auc_knn = knn_clf.fit_predict_score(x, y)
        auc_copod = copod_clf.fit_predict_score(x, y)

        # Добавляем в списки
        auc_list_iforest.append(auc_iforest)
        auc_list_lof.append(auc_lof)
        auc_list_knn.append(auc_knn)
        auc_list_copod.append(auc_copod)

        # Вывод результатов
        print("-" * 50)
        print("Эксперимент №{}/{}  с normal = {}, anomal = {}".format(
            experimant_cnt, all_experiments, c1, c2))
        print("auc_iforest = ", auc_iforest)
        print("auc_lof = ", auc_lof)
        print("auc_knn = ", auc_knn)
        print("auc_copod = ", auc_copod)
        print("-" * 50)


auc_np = np.array(auc_list_iforest)
print("*" * 50)
print("IFOREST Медиана auc = {}".format(np.median(auc_np)))
print("IFOREST Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list_lof)
print("*" * 50)
print("LOF Медиана auc = {}".format(np.median(auc_np)))
print("LOF Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list_knn)
print("*" * 50)
print("kNN Медиана auc = {}".format(np.median(auc_np)))
print("kNN Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list_copod)
print("*" * 50)
print("COPOD Медиана auc = {}".format(np.median(auc_np)))
print("COPOD Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

/Users/pavelmamaev/Desktop/SAIT_DIPLOM/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.8136732596430144


/Users/pavelmamaev/Desktop/SAIT_DIPLOM/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.7947857457474253


/Users/pavelmamaev/Desktop/SAIT_DIPLOM/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


...

roc_auc_score: 0.6556327543424318


/Users/pavelmamaev/Desktop/SAIT_DIPLOM/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.5234739454094293


/Users/pavelmamaev/Desktop/SAIT_DIPLOM/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.48514474772539296
--------------------------------------------------
Эксперимент №56/56  с normal = talk.politics.misc, anomal = soc.religion.christian
auc_iforest =  0.5162117452440034
auc_lof =  0.6556327543424318
auc_knn =  0.5234739454094293
auc_copod =  0.48514474772539296
--------------------------------------------------
**************************************************
IFOREST Медиана auc = 0.5912813590232944
IFOREST Среднее auc = 0.5902735102198052
**************************************************
**************************************************
LOF Медиана auc = 0.6816212121212122
LOF Среднее auc = 0.6668462161005216
**************************************************
**************************************************
kNN Медиана auc = 0.5712815087815089
kNN Среднее auc = 0.5581209225806552
**************************************************
**************************************************
COPOD Медиана auc = 0.539330935333175
COPOD Среднее auc = 0.536622